<a href="https://colab.research.google.com/github/bcrozier/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from re import X

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

features = application_df.drop(columns=['EIN','NAME','IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS'])
features

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT
0,T10,Independent,C1000,ProductDev,Association,1,0,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,5000
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,5000
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,5000
34297,T5,Independent,C3000,ProductDev,Association,1,0,5000


In [53]:
# Determine the number of unique values in each column.
for column in features.columns:
  unique_values = features[column].nunique()
  print(f"{column}: {unique_values}")

APPLICATION_TYPE: 17
AFFILIATION: 6
CLASSIFICATION: 71
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
ASK_AMT: 8747


In [54]:
# Look at APPLICATION_TYPE value counts for binning

application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [55]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace =application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [56]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [57]:
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
classification_counts_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [58]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [59]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_cols = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE",
                    "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"]

application_df_encoded = pd.get_dummies(application_df, columns=categorical_cols)

application_df_encoded


,EIN,NAME,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,1,5000,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,1,108590,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,1,5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,1,6692,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,1,142590,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,1,5000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,1,5000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
34296,996012607,PTA HAWAII CONGRESS,1,5000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,1,5000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [60]:
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL', 'EIN', 'NAME'])
y = application_df_encoded['IS_SUCCESSFUL']

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data and transform both the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Display the shapes of the datasets
print("Training Features Shape:", X_train_scaled.shape)
print("Testing Features Shape:", X_test_scaled.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)

Training Features Shape: (27439, 43)
Testing Features Shape: (6860, 43)
Training Target Shape: (27439,)
Testing Target Shape: (6860,)


In [61]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_features = 43
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=256, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation= 'relu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation= 'relu'))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation= 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 256)               11264     
                                                                 
 dense_26 (Dense)            (None, 128)               32896     
                                                                 
 dense_27 (Dense)            (None, 64)                8256      
                                                                 
 dense_28 (Dense)            (None, 32)                2080      
                                                                 
 dense_29 (Dense)            (None, 16)                528       
                                                                 
 dense_30 (Dense)            (None, 1)                 17        
                                                                 
Total params: 55041 (215.00 KB)
Trainable params: 5504

In [62]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#batch_size=64, validation_data=(X_test_scaled, y_test))

In [63]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5673 - accuracy: 0.7229
Epoch 2/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5546 - accuracy: 0.7289
Epoch 3/10
858/858 [==============================] - 3s 3ms/step - loss: 0.5515 - accuracy: 0.7309
Epoch 4/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5498 - accuracy: 0.7322
Epoch 5/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7320
Epoch 6/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7321
Epoch 7/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7338
Epoch 8/10
858/858 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7339
Epoch 9/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7340
Epoch 10/10
858/858 [==============================] - 2s 3ms/step - loss: 0.5446 - accuracy: 0.7338

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5598 - accuracy: 0.7280 - 643ms/epoch - 3ms/step
Loss: 0.5598427653312683, Accuracy: 0.7279883623123169


In [65]:
# Export our model to HDF5 file
filepath = "AlphabetSoupCharity.h5"
nn.save(filepath)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
